In [104]:
import tensorflow as tf
from tensorflow.keras import layers

# Positional Encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, pos, i, d_model):
        angles = pos / tf.pow(10000, (2 * (i//2)) / tf.cast(d_model, tf.float32))
        return angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        return pos_encoding[tf.newaxis, ...]

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


# Scaled Dot-Product Attention
def scaled_dot_product_attention(q, k, v, mask):              #(Q, K, V는 다 4D 텐서야: (batch_size, num_heads, seq_len, depth))
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights


# Multi-Head Attention
class MultiHeadAttention(layers.Layer):        #num_head = 헤드개수 <예)8개>
    def __init__(self, d_model, num_heads):   #d_model: 전체 출력 차원 (예: 512)
        super().__init__()                     #전체 차원이 head 수로 정확히 나눠떨어져야 해.
        assert d_model % num_heads == 0         #예를 들면 d_model=512, num_heads=8이면 head당 depth는 512/8=64야.

        self.num_heads = num_heads           # 헤드 개수 저장
        self.depth = d_model // num_heads     # 깊이 저장

        self.wq = layers.Dense(d_model)      # Query, Key, Value를 각각 Dense 레이어로 변환할 준비
        self.wk = layers.Dense(d_model)      #각각 입력을 d_model 차원으로 변환시켜.
        self.wv = layers.Dense(d_model)      #여러 head를 만들려면 Q, K, V를 한번 linear transform 해줘야 함. (독립적으로 head들이 일할 수 있도록)

        self.dense = layers.Dense(d_model)   #마지막에 여러 head를 이어붙인 걸 다시 d_model 크기로 맞춰주는 Dense 레이어.

    def split_heads(self, x, batch_size):
        ## 1. (batch_size, seq_len, d_model) → (batch_size, seq_len, num_heads, depth)
        
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))  #x를 (batch_size, seq_len, num_heads, depth)로 reshape.
        # -1은 seq_len 같은 애들이 고정이 아닐 때 자동으로 맞춰주려고 쓰는 거야!
        
        # 2. (batch_size, seq_len, num_heads, depth) → (batch_size, num_heads, seq_len, depth)
        return tf.transpose(x, perm=[0, 2, 1, 3])                        
                                                                         #그리고 num_heads 차원이 앞으로 오게 transpose.
                                                                         #최종 shape: (batch_size, num_heads, seq_len, depth)
                                                                         #멀티헤드니까, head별로 독립적으로 attention을 계산하려고 이렇게 나눔.
                                                                                 
    
    def call(self, v, k, q, attention_mask=None, training=False):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)   #입력 q, k, v에 대해 Dense 통과시켜줌.
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)  #그리고 각각을 멀티헤드 형태로 쪼갬.
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, _ = scaled_dot_product_attention(q, k, v, attention_mask)            #각각의 head에 대해 Scaled Dot-Product Attention 적용.
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])         #q, k, v가 멀티헤드 모양이라 병렬로 적용됨.
    
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.num_heads * self.depth))
        #head들을 하나로 붙임 (concat)
        #shape: (batch_size, seq_len, d_model)

        output = self.dense(concat_attention)  #마지막 Dense를 거쳐서 최종 output 만듦.
        return output
       #[입력 Q, K, V] 
       #  -> (각각 Dense에 통과) 
       #  -> (split_heads로 여러 head로 나눔)
       #  -> (각 head마다 scaled dot-product attention 계산)
       #  -> (head들 concat)
       #  -> (마지막 Dense로 출력)



#-------------------------------- 인코더와 디코더 구조 ------------------------------------#

#📘 Encoder Layer
#[Input] 
# └▶ MultiHeadAttention(Self)
#└▶ Add & Norm
# └▶ Feed Forward
# └▶ Add & Norm
# └▶ Output to next encoder block

#📗 Decoder Layer
#[Previous Output + Encoder Output]
# └▶ MultiHeadAttention(Self + Mask)
# └▶ Add & Norm
# └▶ MultiHeadAttention(from Encoder Output)
# └▶ Add & Norm
# └▶ Feed Forward
# └▶ Add & Norm
# └▶ Output to next decoder block





# Encoder Layer
class EncoderLayer(layers.Layer):                             ## <dff와 rate는 하이퍼파라미터!!>
    def __init__(self, d_model, num_heads, dff, rate=0.1):    ## d_model: 임베딩 차원. 보통 128, 256, 512, 768 같은 2^n 값.
        super().__init__()                                    ## dff: FFN 내부 은닉층의 차원. 보통 d_model * 2 혹은 d_model * 4 정도.
                                                              ##  rate: 드롭아웃 확률. 0.1이면 10%를 랜덤으로 끊어.
        
        self.mha = MultiHeadAttention(d_model, num_heads)  

        self.ffn = tf.keras.Sequential([            ### Feed Forward MLP 은닉층 한개, 출력층 한개. ==> 가중치 줘서 차원 높였다가 줄임. 
            layers.Dense(dff, activation='relu'),   ### 차원을 높였다가 줄이는 이유는 정보 표현 능력을 높이기 위함이다.
            layers.Dense(d_model)
        ])

        ## 층 정규화                                                  #epsilon=1e-6: 분모가 0 되는 걸 방지하는 아주 작은 값.
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)    #미니배치 단위가 아니라 피처 단위로 정규화한다.
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)    #각 위치의 임베딩 벡터 (예: [0.2, -0.4, 1.3, ...]) 를 평균 0, 분산 1로 정규화.


        self.dropout1 = layers.Dropout(rate)                         #드롭아웃 층 두개 
        self.dropout2 = layers.Dropout(rate)


    def call(self, x, mask=None, training=False):                                    
        attn_output = self.mha(x, x, x, attention_mask=mask)                               #1.Self-Attention 수행
        attn_output = self.dropout1(attn_output, training=training)         #2.드롭아웃 ==> 활성화 함수가 없는 은닉층처럼 활용
        out1 = self.layernorm1(x + attn_output)                             #3.잔차연결 및 정규화 <여기서 잔차연결이란 input + attn_output인데 >
 #<여기서 잔차연결이란 input + attn_output인데 정보손실 방지 역할을 한다 그냥 외워>
        
        ffn_output = self.ffn(out1)                                         #4.FFN 통과 ==> 드롭아웃을 거친 뉴런들이 FFN을 통과하면서 다시 비중이 생김
        ffn_output = self.dropout2(ffn_output, training=training)           #5.다시 드롭아웃 => 다시 활성화 함수가 없는 은닉층 처럼 활용
        out2 = self.layernorm2(out1 + ffn_output)                           #6.잔차연결 및 정규화 ==>끝

        return out2


# Decoder Layer
class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask, padding_mask, training):
        attn1 = self.mha1(x, x, x, attention_mask=look_ahead_mask)                      #1단계: Masked Self-Attention
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2 = self.mha2(enc_output, enc_output, out1, attention_mask=padding_mask)    #2단계: Encoder-Decoder Attention
        attn2 = self.dropout2(attn2, training=training)                  #Query: 디코더가 현재까지 만든 출력 (디코더의 상태)
        out2 = self.layernorm2(attn2 + out1)                             #Key, Value: 인코더의 출력 (전체 문장의 의미 벡터들)
                                                                         #이걸로 어텐션 → context vector를 구함
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3


#<디코더 레이어 전체 그림>

#Input x
#  ↓
#Masked MultiHead Attention (Q=K=V=x, look-ahead mask)
#  ↓
#Residual + LayerNorm
#  ↓
#MultiHead Attention with Encoder Output (Q=Decoder, K=V=Encoder)
#  ↓
#Residual + LayerNorm
#  ↓
#Feed Forward Network (Dense → ReLU → Dense)
#  ↓
#Residual + LayerNorm
#  ↓
#Output (x의 디코더 처리 결과)






#항목	     Encoder의 Multi-Head Attention	          vs               Decoder의 Masked Multi-Head Self-Attention

#목적	     입력 전체를 스스로 이해 (글의 의미 파악)	                   이전 단어만 보고 다음 단어 예측
#마스크	     Padding mask만 사용	                                       Padding mask + Look-ahead mask 사용
#정보 접근	 모든 토큰에 자유롭게 접근	                               자기 앞의 단어까지만 접근 가능 (미래 차단)





#-----------------------------------------------------------------------------------------------------------------------------------------------------

# Encoder
class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                                                ##maximum_position_encoding : 입력 문장의 최대 길이를 의미해!

        self.d_model = d_model
        self.num_layers = num_layers     ## 인코딩 층을 얼마나 통과시킬 건지 => 보통 6개를 많이 쓴다고 한다.

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))   
        #이건 임베딩된 벡터의 스케일을 조정해주는 코드야. 멀티헤드어텐션과 다른 점은 스케일을 키워준다!!
        
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for enc_layer in self.enc_layers:           ##
            x = enc_layer(x, mask=mask, training=training)

        return x        #최종 출력 x는 크기가 (batch_size, input_seq_len, d_model)인 텐서
                        #이걸 디코더에서 키-값 벡터로 참조해서 예측에 사용해 ==>> 인코더-디코더 어텐션에서의 KEY, VALUE로 input되는 값이 된다.









# 100자 이내의 짧은 문장만 다룬다면
#maximum_position_encoding = 100

# 문장 번역용 일반 Transformer라면
#maximum_position_encoding = 512

# 논문 전체처럼 긴 문서를 처리할 거라면
#maximum_position_encoding = 1024 ~ 4096

# Decoder
class Decoder(layers.Layer):            #target_vocab_size : 예측할 대상 단어 집합의 크기  ///  maximum_position_encoding	: 위치 인코딩에서 최대 문장길이
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                 #dff => FFN 은닉층 차원 

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(target_vocab_size, d_model)   
        #디코더의 embedding()은 **"정답 문장"의 앞부분(예측해야 할 문장의 이전 단어들)**을 임베딩하는 거야.

#단계	                     입력값	                        설명
#Encoder input	             "나는 밥을 먹었다"	            인코더에 들어갈 입력 문장 (source)
#Decoder input	             ["<s>", "I", "ate"]	        디코더에 들어갈 입력 (target의 앞부분)
#Decoder output (target)	 ["I", "ate", "rice"]	        디코더가 예측해야 하는 실제 정답
       
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)   

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask=None, padding_mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for dec_layer in self.dec_layers:
            x = dec_layer(x, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask, training=training)

        return x   #(batch_size, target_seq_len, target_vocab_size)   #디코더 각 위치에서 최종적으로 만들어낸 벡터들



# Transformer

class Transformer(tf.keras.Model):   
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super().__init__()   
        #input_vocab_size:인코더에 들어갈 문장의 단어 집합 크기 (vocabulary size)
        #target_vocab_size : 디코더가 생성할 문장의 단어 집합 크기
        #pe_input : Positional Encoding을 위한 입력 시퀀스 최대 길이, 예: 입력 문장의 최대 길이가 100이라면 → pe_input = 100
        #pe_target : 디코더 입력 (정답 문장)의 최대 길이, 예: : 정답 문장의 최대 길이가 80이라면 → pe_target = 80
        #주의: pe_input, pe_target은 학습 데이터보다 충분히 크게 잡자! 최대 문장 길이보다 살짝 크게 잡아두는 게 안전함 (ex. 128, 256 등)
        
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
        self.final_layer = layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        # unpacking
        inp, tar = inputs

        # 마스크 생성
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)

        # 인코더-디코더 흐름
        enc_output = self.encoder(inp, mask=enc_padding_mask, training=training)
        dec_output = self.decoder(tar, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask, training=training)
        final_output = self.final_layer(dec_output)

        return final_output
        #로짓 (logits): 아직 softmax는 안 씌운 생 확률 점수들. 
        #보통 loss 계산할 때 softmax 없이 logits을 쓰는 게 더 안정적이기 때문에 이렇게 반환해.





        

#Input Sentence  --->  Encoder ----------------------------┐
#                    ↓                                     ↓
#               Encoded Context  -->  Decoder (with target input)
#                                          ↓
#                                 Final Dense (Softmax logits)




#"입력 토큰 ID" = 정수 인코딩된 값





# 요약 흐름
#입력문장 inp → 인코더에 들어감 → enc_output 생성

#정답 문장 tar → 디코더에 들어감 (enc_output 참고함)

#디코더 출력 dec_output → Dense layer → 단어 확률 분포 final_output

#리턴: (batch, target_seq_len, vocab_size) 형태의 로짓

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Positional Encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, pos, i, d_model):
        angles = pos / tf.pow(10000, (2 * (i//2)) / tf.cast(d_model, tf.float32))
        return angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        return pos_encoding[tf.newaxis, ...]

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

def scaled_dot_product_attention(q, k, v, mask):              
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

class MultiHeadAttention(layers.Layer):       
    def __init__(self, d_model, num_heads):   
        super().__init__()                     
        assert d_model % num_heads == 0        

        self.num_heads = num_heads           
        self.depth = d_model // num_heads   

        self.wq = layers.Dense(d_model)     
        self.wk = layers.Dense(d_model)      
        self.wv = layers.Dense(d_model)     

        self.dense = layers.Dense(d_model)   

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))  
        return tf.transpose(x, perm=[0, 2, 1, 3]) 

    def call(self, v, k, q, attention_mask=None, training=False):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)   #입력 q, k, v에 대해 Dense 통과시켜줌.
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)  #그리고 각각을 멀티헤드 형태로 쪼갬.
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, _ = scaled_dot_product_attention(q, k, v, attention_mask)            
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])         
    
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.num_heads * self.depth))
        output = self.dense(concat_attention)  
        return output

class EncoderLayer(layers.Layer):                           
    def __init__(self, d_model, num_heads, dff, rate=0.1):    
        super().__init__()                                    
                                                              
        
        self.mha = MultiHeadAttention(d_model, num_heads)  

        self.ffn = tf.keras.Sequential([          
            layers.Dense(dff, activation='relu'),   
            layers.Dense(d_model)
        ])

        ## 층 정규화                                                  
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)    
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)    


        self.dropout1 = layers.Dropout(rate)                         
        self.dropout2 = layers.Dropout(rate)


    def call(self, x, mask=None, training=False):                                    
        attn_output = self.mha(x, x, x, attention_mask=mask)                               
        attn_output = self.dropout1(attn_output, training=training)        
        out1 = self.layernorm1(x + attn_output)                             
        
        ffn_output = self.ffn(out1)                                         
        ffn_output = self.dropout2(ffn_output, training=training)           
        out2 = self.layernorm2(out1 + ffn_output)                           

        return out2


class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask, padding_mask, training):
        attn1 = self.mha1(x, x, x, attention_mask=look_ahead_mask)                 
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2 = self.mha2(enc_output, enc_output, out1, attention_mask=padding_mask)    
        attn2 = self.dropout2(attn2, training=training)                  
        out2 = self.layernorm2(attn2 + out1)                             
                                                                         
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3

class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                                             

        self.d_model = d_model
        self.num_layers = num_layers 

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))   
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for enc_layer in self.enc_layers:           
            x = enc_layer(x, mask=mask, training=training)

        return x  

class Decoder(layers.Layer):            
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                 

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(target_vocab_size, d_model)        
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)   
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask=None, padding_mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for dec_layer in self.dec_layers:
            x = dec_layer(x, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask, training=training)

        return x

class Transformer(tf.keras.Model):   
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super().__init__()   
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
        self.final_layer = layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        inp, tar = inputs
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
        enc_output = self.encoder(inp, mask=enc_padding_mask, training=training)
        dec_output = self.decoder(tar, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask, training=training)
        final_output = self.final_layer(dec_output)

        return final_output

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Positional Encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, pos, i, d_model):
        angles = pos / tf.pow(10000, (2 * (i//2)) / tf.cast(d_model, tf.float32))
        return angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        return pos_encoding[tf.newaxis, ...]

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

def scaled_dot_product_attention(q, k, v, mask):              
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

class MultiHeadAttention(layers.Layer):       
    def __init__(self, d_model, num_heads):   
        super().__init__()                     
        assert d_model % num_heads == 0        

        self.num_heads = num_heads           
        self.depth = d_model // num_heads   

        self.wq = layers.Dense(d_model)     
        self.wk = layers.Dense(d_model)      
        self.wv = layers.Dense(d_model)     

        self.dense = layers.Dense(d_model)   

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))  
        return tf.transpose(x, perm=[0, 2, 1, 3]) 

    def call(self, v, k, q, attention_mask=None, training=False):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)   #입력 q, k, v에 대해 Dense 통과시켜줌.
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)  #그리고 각각을 멀티헤드 형태로 쪼갬.
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, _ = scaled_dot_product_attention(q, k, v, attention_mask)            
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])         
    
        
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.num_heads * self.depth))
        output = self.dense(concat_attention)  
        return output

class EncoderLayer(layers.Layer):                           
    def __init__(self, d_model, num_heads, dff, rate=0.1):    
        super().__init__()                                    
                                                              
        
        self.mha = MultiHeadAttention(d_model, num_heads)  

        self.ffn = tf.keras.Sequential([          
            layers.Dense(dff, activation='relu'),   
            layers.Dense(d_model)
        ])

        ## 층 정규화                                                  
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)    
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)    


        self.dropout1 = layers.Dropout(rate)                         
        self.dropout2 = layers.Dropout(rate)


    def call(self, x, mask=None, training=False):                                    
        attn_output = self.mha(x, x, x, attention_mask=mask)                               
        attn_output = self.dropout1(attn_output, training=training)        
        out1 = self.layernorm1(x + attn_output)                             
        
        ffn_output = self.ffn(out1)                                         
        ffn_output = self.dropout2(ffn_output, training=training)           
        out2 = self.layernorm2(out1 + ffn_output)                           

        return out2


class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask, padding_mask, training):
        attn1 = self.mha1(x, x, x, attention_mask=look_ahead_mask)                 
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2 = self.mha2(enc_output, enc_output, out1, attention_mask=padding_mask)    
        attn2 = self.dropout2(attn2, training=training)                  
        out2 = self.layernorm2(attn2 + out1)                             
                                                                         
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3

class Encoder(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                                             

        self.d_model = d_model
        self.num_layers = num_layers 

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))   
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for enc_layer in self.enc_layers:           
            x = enc_layer(x, mask=mask, training=training)

        return x  

class Decoder(layers.Layer):            
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super().__init__()                 

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(target_vocab_size, d_model)        
        self.pos_encoding = PositionalEncoding(maximum_position_encoding, d_model)   
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)

    def call(self, x, enc_output, look_ahead_mask=None, padding_mask=None, training=False):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = self.pos_encoding(x)
        x = self.dropout(x, training=training)

        for dec_layer in self.dec_layers:
            x = dec_layer(x, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask, training=training)

        return x

class Transformer(tf.keras.Model):   
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super().__init__()   
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
        self.final_layer = layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        inp, tar = inputs
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
        enc_output = self.encoder(inp, mask=enc_padding_mask, training=training)
        dec_output = self.decoder(tar, enc_output, look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask, training=training)
        final_output = self.final_layer(dec_output)

        return final_output

In [ ]:
tar 에 관한 것
상황	       디코더 입력	                        예측 방식	                Mask
훈련	    ["<start>", "I", "love", "you"]	        병렬로 전체 예측	            ✅ Look-ahead mask 사용
추론(예측)	["<start>"] → ["<start>", "I"] → ...	순차적으로 한 토큰씩 예측	    ✅ Look-ahead mask 사용


In [2]:
# 여기까지 트랜스포머를 이해하기 위한 미니트랜스포머의 구현이 끝났다.
# 트랜스포머 구현을 했고 이를 이해하기 위해 노력했지만, 아마 완전히 이해하지는 못했을 것이다.
# 이는 수많은 연습과 나만의 프로젝트를 구현하면서 극복할 것으로 예상된다.
# 토스 ceo가 열심히 노력하면 언젠간 기회는 찾아온다고 했다. 난 그 기회를 잡기 위해서 열심히 노력하고 있는 것이다.

In [ ]:
#트랜스포머 기반 챗봇

✅ 챗봇 구현 흐름 (Transformer 기반)
1.데이터 준비

  질문과 답변 쌍으로 구성된 대화 데이터셋 (ex. “졸업 요건이 뭐야?” → “총 130학점이 필요해요”)

  정수 인코딩 (tokenizer 사용)

2.입출력 데이터 전처리

  패딩 / 마스킹 / 시작 토큰 (<start>) / 종료 토큰 (<end>) 붙이기

  inp → encoder 입력

  tar_input → decoder 입력

  tar_real → decoder 출력(정답)

3.트랜스포머 모델 구현

  방금 만든 Transformer 클래스 사용

  loss function, optimizer 정의

4.모델 학습

  teacher forcing 기법 적용

  여러 epoch 동안 학습

5.챗봇 예측 (inference)

  하나의 질문 입력 → decoder가 한 글자씩 자동으로 다음 단어 예측 (auto-regressive)

  <end> 토큰이 나올 때까지 반복

6.CLI 기반 챗봇 만들기

  사용자 입력 받기 → 모델 응답 출력

In [10]:
#데이터 전처리를 해보자.
import pandas as pd
import re
df = pd.read_csv("gradu_qna_dummy.csv", encoding = 'euc-kr')
#print(df)

# Drop rows with missing Q or A
df.dropna(subset=["Q", "A"], inplace=True)  # inplace=True  ==> 복사본 만들지 말고 원본으로

# Basic text cleaning function
def clean_text(text):
    text = str(text)
    text = re.sub(r"[^가-힣0-9a-zA-Z\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text)  # Normalize whitespace
    return text.strip()

# Apply cleaning
df["Q_clean"] = df["Q"].apply(clean_text)
df["A_clean"] = df["A"].apply(clean_text)

# Show a sample
df[["Q", "Q_clean", "A", "A_clean"]].head()

#questions = df['Q'].astype(str).tolist()
#answers = df['A'].astype(str).tolist()



,Q,Q_clean,A,A_clean
0,졸업요건 알려줘,졸업요건 알려줘,졸업요건을 알려면 학번과 학과를 알려줘.,졸업요건을 알려면 학번과 학과를 알려줘
1,어학 기준이 궁금해,어학 기준이 궁금해,어학 기준은 학번별로 달라. 학번을 알려줄래?,어학 기준은 학번별로 달라 학번을 알려줄래
2,논문 꼭 써야 해?,논문 꼭 써야 해,논문 대신 졸업 프로젝트나 포트폴리오로 대체할 수 있어.,논문 대신 졸업 프로젝트나 포트폴리오로 대체할 수 있어
3,총 몇 학점 들어야 해?,총 몇 학점 들어야 해,총 130학점 이상 이수해야 졸업할 수 있어.,총 130학점 이상 이수해야 졸업할 수 있어
4,교양필수 과목 알려줘,교양필수 과목 알려줘,"대학공통교양으로는 글쓰기, 영어, 컴퓨팅사고가 포함돼.",대학공통교양으로는 글쓰기 영어 컴퓨팅사고가 포함돼


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

train_data = pd.read_csv("gradu_qna_dummy.csv", encoding = 'euc-kr')

questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []
for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

print(questions[:5])
print(answers[:5])


['졸업요건 알려줘', '어학 기준이 궁금해', '논문 꼭 써야 해 ?', '총 몇 학점 들어야 해 ?', '교양필수 과목 알려줘']
['졸업요건을 알려면 학번과 학과를 알려줘 .', '어학 기준은 학번별로 달라 .  학번을 알려줄래 ?', '논문 대신 졸업 프로젝트나 포트폴리오로 대체할 수 있어 .', '총 130학점 이상 이수해야 졸업할 수 있어 .', '대학공통교양으로는 글쓰기 ,  영어 ,  컴퓨팅사고가 포함돼 .']


In [14]:
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)


# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
#리스트 형태로 만들어둔 이유는 나중에 input + START_TOKEN + END_TOKEN 같이 쉽게 붙이기 위해서.


# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

print(questions[20])

시작 토큰 번호 : [416]
종료 토큰 번호 : [417]
단어 집합의 크기 : 418
나는 2021학번이고 데이터사이언스학부야


In [16]:
#정수 인코딩과 패딩 시험용

# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))


# 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# 임의의 입력 문장을 sample_string에 저장
sample_string = questions[20]

# encode() : 텍스트 시퀀스 --> 정수 시퀀스
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode() : 정수 시퀀스 --> 텍스트 시퀀스
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}\n'.format(original_string))
# 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# 임의의 입력 문장을 sample_string에 저장


# 각 정수는 각 단어와 어떻게 mapping되는지 병렬로 출력
# 서브워드텍스트인코더는 의미있는 단위의 서브워드로 토크나이징한다. 띄어쓰기 단위 X 형태소 분석 단위 X
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))


임의의 질문 샘플을 정수 인코딩 : [23, 80, 69]
정수 인코딩 후의 문장 [23, 80, 69]
기존 문장: 나는 2021학번이고 데이터사이언스학부야

23 ----> 나는 
80 ----> 2021학번이고 
69 ----> 데이터사이언스학부야


In [18]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs



questions, answers = tokenize_and_filter(questions, answers)


print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)





질문 데이터의 크기(shape) : (95, 40)
답변 데이터의 크기(shape) : (95, 40)


In [20]:
# 디코더의 입력은 답변 시퀀스에서 마지막 토큰을 제외한 것
decoder_inputs = answers[:, :-1]

# 디코더의 정답은 첫 번째 토큰을 제외한 것 (한 칸 오른쪽으로 shift)
decoder_targets = answers[:, 1:]

#encoder_input, decoder_input, decoder_target 이 트리플 세트로 묶여서, 각 문장(샘플)마다 하나씩 학습이 진행되는 구조야.


dataset = tf.data.Dataset.from_tensor_slices((
    (questions, decoder_inputs),  # input = (encoder_input, decoder_input)
    decoder_targets
))

BATCH_SIZE = 64
BUFFER_SIZE = 10000  #일반적으로는 데이터 전체 수 이상을 쓰는 게 좋아!

dataset = dataset.shuffle(BUFFER_SIZE)  #데이터를 무작위로 섞어서 과적합을 줄이고 일반화 성능을 높임.
dataset = dataset.batch(BATCH_SIZE)     #BATCH_SIZE 단위로 데이터를 묶어 모델에 전달
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)   #학습 중에 다음 데이터를 미리 준비해서 GPU가 쉬지 않게 함





# 1. 인코더가 질문을 보고 문맥 파악
# encoder_output = encoder(encoder_input)

# 2. 디코더가 decoder_input을 보고 다음 단어를 예측
# decoder_output = decoder(decoder_input, encoder_output)

# 3. decoder_output을 final dense layer 통과 → 예측값들 (확률 분포)
#    예측값 vs decoder_target 비교 → loss 계산 → backpropagation


In [22]:
transformer = Transformer(
    num_layers=2,
    d_model=128,
    num_heads=4,
    dff=512,
    input_vocab_size=VOCAB_SIZE,
    target_vocab_size=VOCAB_SIZE,
    pe_input=MAX_LENGTH,
    pe_target=MAX_LENGTH,
    rate=0.1
)


In [39]:
EPOCHS = 200  # 원하는 만큼 반복


def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask






class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        # 학습률 계산 공식
        
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)





loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    # 패딩 마스크
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask  # 패딩 제외한 토큰만 계산

    return tf.reduce_mean(loss_)

learning_rate = CustomSchedule(128)  # 네 모델에서 d_model 사용했을 거야
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy_function(real, pred):
    real = tf.cast(real, tf.int64)
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)

    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

transformer.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])

transformer.fit(dataset, epochs=EPOCHS)



Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 194ms/step - accuracy_function: 0.9684 - loss: 0.0989
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy_function: 0.9673 - loss: 0.0900
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy_function: 0.9727 - loss: 0.0995
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy_function: 0.9718 - loss: 0.0997
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy_function: 0.9735 - loss: 0.0964
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step - accuracy_function: 0.9712 - loss: 0.1026
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy_function: 0.9700 - loss: 0.0954
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy_function: 0.9674 - loss: 0.0957
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy_function: 0.9734 - loss: 0.0937
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy_function: 0.9726 - loss: 0.0972
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accu

In [4]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = transformer(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)

def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence


In [6]:
output = predict("안녕")


NameError: name 're' is not defined